<a href="https://colab.research.google.com/github/ChoeTaeBin/Machine-Learnig/blob/main/digdata_HakerTone_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

In [5]:
!pip install schedule

In [9]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
import schedule
import time
import pandas as pd
import urllib.request
import json

conn = sqlite3.connect('bigdata_DB')

client_id = "8sAydq_QSOxKcICuD2Og"
client_secret = "OruQjbmaXF"

# 테이블 생성 함수
def create_tables():
    with conn.cursor() as curs:
        sql = """
        CREATE TABLE IF NOT EXISTS stock_prices (
            id INT AUTO_INCREMENT PRIMARY KEY,
            code VARCHAR(20),
            date DATETIME,
            open BIGINT,
            high BIGINT,
            low BIGINT,
            close BIGINT,
            volume BIGINT
        )
        """
        curs.execute(sql)

        sql = """
        CREATE TABLE IF NOT EXISTS news (
            id INT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(255),
            description TEXT,
            org_link TEXT,
            link TEXT,
            pubDate DATETIME,
            keyword VARCHAR(50)
        )
        """
        curs.execute(sql)
    conn.commit()

create_tables()

# 네이버에서 실시간 주식 시세를 읽어서 반환하는 함수
def read_naver_timely(code, company):
    try:
        KST = timezone(timedelta(hours=9))
        now = datetime.now(KST).strftime('%Y%m%d%H%M%S')
        url = f"https://finance.naver.com/item/sise_time.naver?code={code}&thistime={now}"
        df = pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)[0]
        df = df.dropna()
        df["date"] = pd.to_datetime(df['체결시간'].apply(lambda x: now[:8] + x.replace(':', '')))
        df.rename(columns={'체결가': 'close', '거래량': 'volume', '시가': 'open', '고가': 'high', '저가': 'low'}, inplace=True)
        df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
    except Exception as e:
        print('Exception occurred:', str(e))
        return None
    return df

# 주식 데이터를 DB에 저장하는 함수
def replace_into_db_min(df, code, company):
    with conn.cursor() as curs:
        for r in df.itertuples():
            sql = f"REPLACE INTO stock_prices (code, date, open, high, low, close, volume) VALUES ('{code}', '{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, {r.volume})"
            curs.execute(sql)
        conn.commit()
        print(f'[{datetime.now().strftime("%Y-%m-%d %H:%M")}] {company} ({code}) : {len(df)} rows > REPLACE INTO stock_prices [OK]')

# 뉴스 데이터를 요청하는 함수
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-ID", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

# 네이버 검색 API를 호출하는 함수
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    url = base + node + parameters
    responseDecode = getRequestUrl(url)
    if responseDecode is None:
        return None
    else:
        return json.loads(responseDecode)

# 뉴스 데이터를 DB에 저장하는 함수
def getPostData(post, keyword):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    with conn.cursor() as curs:
        sql = """
        INSERT INTO news (title, description, org_link, link, pubDate, keyword)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        curs.execute(sql, (title, description, org_link, link, pDate, keyword))
    conn.commit()

# 뉴스 크롤링 함수
def crawl_news(keyword):
    node = 'news'
    jsonResponse = getNaverSearch(node, keyword, 1, 100)

    if jsonResponse is not None:
        while (jsonResponse is not None and jsonResponse['display'] != 0):
            for post in jsonResponse['items']:
                getPostData(post, keyword)
            start = jsonResponse['start'] + jsonResponse['display']
            jsonResponse = getNaverSearch(node, keyword, start, 100)
        print("데이터 저장 완료.")
    else:
        print("No Response")

# 주식 및 뉴스 크롤링 작업
def job():
    # 주식 코드 및 회사명 설정 (예: 넷마블)
    stock_code = '251270'
    company_name = '넷마블'

    # 주가 정보 크롤링 및 DB 저장
    df_stock = read_naver_timely(stock_code, company_name)
    if df_stock is not None:
        replace_into_db_min(df_stock, stock_code, company_name)

    # 뉴스 정보 크롤링 및 DB 저장
    keyword = '넷마블'  # 검색 키워드 설정
    crawl_news(keyword)

# # 스케줄러 설정 (매 1분마다 실행)
# schedule.every(1).minutes.do(job)

# print("주식 및 뉴스 크롤러가 실행됩니다.")

# # 스케줄러 실행
# while True:
#     schedule.run_pending()
#     time.sleep(1)

# 스케줄러 설정 (10초마다 실행)
schedule.every(10).seconds.do(job)

# 스케줄러 실행
start_time = datetime.now()
end_time = start_time + timedelta(minutes=1)

while datetime.now() < end_time:
    schedule.run_pending()
    time.sleep(1)


AttributeError: __enter__

In [36]:
import sqlite3
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
import schedule
import time
import pandas as pd
import urllib.request
import json

conn = sqlite3.connect('bigdata_DB')

client_id = 'IwiCHTYaP72gNDJZzLKn'
client_secret = '5WhbnXIWyI'

# 테이블 생성 함수
def create_tables():
    curs = conn.cursor()
    try:
        sql = """
        CREATE TABLE IF NOT EXISTS stock_prices (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            code TEXT,
            time DATETIME,
            close BIGINT,
            before BIGINT,
            sell BIGINT,
            buy BIGINT,
            amount BIGINT
            change BIGINT
        )
        """
        curs.execute(sql)

        sql = """
        CREATE TABLE IF NOT EXISTS news (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            description TEXT,
            org_link TEXT,
            link TEXT,
            pubDate DATETIME,
            keyword TEXT
        )
        """
        curs.execute(sql)
    finally:
        curs.close()
    conn.commit()

create_tables()

# 네이버에서 실시간 주식 시세를 읽어서 반환하는 함수
def read_naver_timely(code, company):
    try:
        KST = timezone(timedelta(hours=9))
        now = datetime.now(KST).strftime('%Y%m%d%H%M%S')
        url = f"https://finance.naver.com/item/sise_time.naver?code={code}&thistime={now}"
        df = pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)[0]
        df = df.dropna()
        df["time"] = pd.to_datetime(df['체결시각'].apply(lambda x: now[:8] + x.replace(':', '')))
        df.rename(columns={'체결가': 'close', '전일비': 'before', '매도': 'sell', '매수': 'buy', '거래량': 'amount', '변동량': 'change'}, inplace=True)
        df = df[['time', 'close', 'before', 'sell', 'buy', 'amount', 'change']] ##
    except Exception as e:
        print('Exception occurred:', str(e))
        return None
    return df

# 주식 데이터를 DB에 저장하는 함수
def replace_into_db_min(df, code, company):
    curs = conn.cursor()
    try:
        for r in df.itertuples():
            sql = f"REPLACE INTO stock_prices (code, time, close, before, sell, buy, amount, change) VALUES ('{code}', '{r.time}', {r.close}, {r.before}, {r.sell}, {r.buy}, {r.amount}, {r.change})"
            curs.execute(sql)
        conn.commit()
        print(f'[{datetime.now().strftime("%Y-%m-%d %H:%M")}] {company} ({code}) : {len(df)} rows > REPLACE INTO stock_prices [OK]')
    finally:
        curs.close()

# 뉴스 데이터를 요청하는 함수
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-ID", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.now(), url))
        return None

# 네이버 검색 API를 호출하는 함수
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    url = base + node + parameters
    responseDecode = getRequestUrl(url)
    if responseDecode is None:
        return None
    else:
        return json.loads(responseDecode)

# 뉴스 데이터를 DB에 저장하는 함수
def getPostData(post, keyword):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    pDate = datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    curs = conn.cursor()
    try:
        sql = """
        INSERT INTO news (title, description, org_link, link, pubDate, keyword)
        VALUES (?, ?, ?, ?, ?, ?)
        """
        curs.execute(sql, (title, description, org_link, link, pDate, keyword))
    finally:
        curs.close()
    conn.commit()

# 뉴스 크롤링 함수
def crawl_news(keyword):
    node = 'news'
    jsonResponse = getNaverSearch(node, keyword, 1, 100)

    if jsonResponse is not None:
        while (jsonResponse is not None and jsonResponse['display'] != 0):
            for post in jsonResponse['items']:
                getPostData(post, keyword)
            start = jsonResponse['start'] + jsonResponse['display']
            jsonResponse = getNaverSearch(node, keyword, start, 100)
        print("데이터 저장 완료.")
    else:
        print("No Response")

# 주식 및 뉴스 크롤링 작업
def job():
    # 주식 코드 및 회사명 설정 (예: 넷마블)
    stock_code = '012450'
    company_name = '한화에어로스페이스'

    # 주가 정보 크롤링 및 DB 저장
    df_stock = read_naver_timely(stock_code, company_name)
    if df_stock is not None:
        replace_into_db_min(df_stock, stock_code, company_name)

    # 뉴스 정보 크롤링 및 DB 저장
    keyword = '북한'  # 검색 키워드 설정
    crawl_news(keyword)

# 스케줄러 설정 (10초마다 실행)
schedule.every(10).seconds.do(job)

# 스케줄러 실행
start_time = datetime.now()
end_time = start_time + timedelta(minutes=1)

while datetime.now() < end_time:
    schedule.run_pending()
    time.sleep(1)

print("1분이 경과하여 스케줄러를 종료합니다.")

# SQLite 데이터베이스 연결 닫기
conn.close()


OperationalError: near "1": syntax error

In [41]:
import sqlite3
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
import schedule
import time
import pandas as pd
import urllib.request
import json

conn = sqlite3.connect('bigdata_DB')

client_id = 'IwiCHTYaP72gNDJZzLKn'
client_secret = '5WhbnXIWyI'

# 테이블 생성 함수
def create_tables():
    curs = conn.cursor()
    try:
        sql = """
        CREATE TABLE IF NOT EXISTS stock_prices (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            code TEXT,
            time DATETIME,
            close BIGINT,
            before BIGINT,
            sell BIGINT,
            buy BIGINT,
            amount BIGINT,
            change BIGINT
        )
        """
        curs.execute(sql)

        sql = """
        CREATE TABLE IF NOT EXISTS news (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            description TEXT,
            org_link TEXT,
            link TEXT,
            pubDate DATETIME,
            keyword TEXT
        )
        """
        curs.execute(sql)
    finally:
        curs.close()
    conn.commit()

create_tables()


# 네이버에서 실시간 주식 시세를 읽어서 반환하는 함수
def read_naver_timely(code, company):
    try:
        KST = timezone(timedelta(hours=9))
        now = datetime.now(KST).strftime('%Y%m%d%H%M%S')
        url = f"https://finance.naver.com/item/sise_time.naver?code={code}&thistime={now}"
        df = pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)[0]
        df = df.dropna()
        df["time"] = pd.to_datetime(df['체결시각'].apply(lambda x: now[:8] + x.replace(':', '')))
        df.rename(columns={'체결가': 'close', '전일비': 'before', '매도': 'sell', '매수': 'buy', '거래량': 'amount', '변동량': 'change'}, inplace=True)
        df = df[['time', 'close', 'before', 'sell', 'buy', 'amount', 'change']]  # 컬럼 선택 수정
    except Exception as e:
        print('Exception occurred:', str(e))
        return None
    return df

# 주식 데이터를 DB에 저장하는 함수
def replace_into_db_min(df, code, company):
    curs = conn.cursor()
    try:
        for r in df.itertuples():
            sql = "REPLACE INTO stock_prices (code, time, close, before, sell, buy, amount, change) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
            curs.execute(sql, (code, r.time, r.close, r.before, r.sell, r.buy, r.amount, r.change))
        conn.commit()
        print(f'[{datetime.now().strftime("%Y-%m-%d %H:%M")}] {company} ({code}) : {len(df)} rows > REPLACE INTO stock_prices [OK]')
    finally:
        pass  # 예외 발생 여부와 상관없이 커서를 닫지 않음


# 뉴스 데이터를 요청하는 함수
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-ID", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.now(), url))
        return None

# 네이버 검색 API를 호출하는 함수
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    url = base + node + parameters
    responseDecode = getRequestUrl(url)
    if responseDecode is None:
        return None
    else:
        return json.loads(responseDecode)

# 뉴스 데이터를 DB에 저장하는 함수
def getPostData(post, keyword):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    pDate = datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    curs = conn.cursor()
    try:
        sql = """
        INSERT INTO news (title, description, org_link, link, pubDate, keyword)
        VALUES (?, ?, ?, ?, ?, ?)
        """
        curs.execute(sql, (title, description, org_link, link, pDate, keyword))
    finally:
        pass  # 수정: 예외 발생 여부와 상관없이 커서를 닫지 않음
    conn.commit()

# 뉴스 크롤링 함수
def crawl_news(keyword):
    node = 'news'
    jsonResponse = getNaverSearch(node, keyword, 1, 100)

    if jsonResponse is not None:
        while (jsonResponse is not None and jsonResponse['display'] != 0):
            for post in jsonResponse['items']:
                getPostData(post, keyword)
            start = jsonResponse['start'] + jsonResponse['display']
            jsonResponse = getNaverSearch(node, keyword, start, 100)
        print("데이터 저장 완료.")
    else:
        print("No Response")

# 주식 및 뉴스 크롤링 작업
def job():
    # 주식 코드 및 회사명 설정 (예: 넷마블)
    stock_code = '012450'
    company_name = '한화에어로스페이스'

    # 주가 정보 크롤링 및 DB 저장
    df_stock = read_naver_timely(stock_code, company_name)
    if df_stock is not None:
        replace_into_db_min(df_stock, stock_code, company_name)

    # 뉴스 정보 크롤링 및 DB 저장
    keyword = '북한'  # 검색 키워드 설정
    crawl_news(keyword)

# 스케줄러 설정 (10초마다 실행)
schedule.every(10).seconds.do(job)

# 스케줄러 실행
start_time = datetime.now()
end_time = start_time + timedelta(minutes=1)

while datetime.now() < end_time:
    schedule.run_pending()
    time.sleep(1)

print("1분이 경과하여 스케줄러를 종료합니다.")

# SQLite 데이터베이스 연결 닫기
conn.close()


OperationalError: table stock_prices has no column named time

In [23]:
conn = sqlite3.connect('bigdata_DB')

In [28]:
conn.close()

In [24]:

# SQL 쿼리 작성 (예: 'SELECT * FROM your_table')
query = 'SELECT * FROM news'

# SQL 쿼리를 실행하고 결과를 DataFrame으로 변환
df = pd.read_sql_query(query, conn)


In [25]:
df

,id,title,description,org_link,link,pubDate,keyword
0,1,"<b>넷마블</b>, 29일 기대작 레이븐2 국내 정식 출시…정통 다크 판타지 콘셉트",구글플레이·애플 앱스토어·PC 플랫폼 통해...총 30개 서버 오픈 <b>넷마블</...,https://www.nspna.com/news/?mode=view&newsid=7...,https://www.nspna.com/news/?mode=view&newsid=7...,2024-05-29 12:02:00,넷마블
1,2,"<b>넷마블</b> 신작 MMORPG '레이븐2', 29일 오후 8시 정식 출시","서비스 ‘<b>넷마블</b> 커넥트’ 앱 제공, 모바일-PC 원격 플레이 가능- 인...",https://www.thisisgame.com/webzine/game/nboard...,https://www.thisisgame.com/webzine/game/nboard...,2024-05-29 11:58:00,넷마블
2,3,"넥슨 메이플스토리, 블록체인 기술 융합 스토리 대공개",대한민국 NFT/블록체인 게임 컨퍼런스 현장에서는 넥산 넥스페이스 외에도 ▲홍진표 ...,https://zdnet.co.kr/view/?no=20240529115503,https://n.news.naver.com/mnews/article/092/000...,2024-05-29 11:56:00,넷마블
3,4,"'레이븐2', 오늘(29일) 오후 8시 정식 출시","<b>넷마블</b>(대표 권영식, 김병규)은 블록버스터 신작 MMORPG '레이븐2...",https://www.inven.co.kr/webzine/news/?news=296033,https://m.sports.naver.com/esports/article/442...,2024-05-29 11:52:00,넷마블
4,5,"<b>넷마블</b> 블록버스터 신작 ‘레이븐2’, 오늘 정식 출시",한국정경신문 임윤희 기­자 yunis1004@naver.com <b>넷마블</b>이...,http://kpenews.com/View.aspx?No=3247539,http://kpenews.com/View.aspx?No=3247539,2024-05-29 11:50:00,넷마블
...,...,...,...,...,...,...,...
1995,1996,"레이븐2, 드디어 나온다","<b>넷마블</b>(대표 권영식, 김병규)은 블록버스터 신작 MMORPG '레이븐2...",https://www.econovill.com/news/articleView.htm...,https://www.econovill.com/news/articleView.htm...,2024-05-22 11:48:00,넷마블
1996,1997,<b>넷마블</b> 신작 MMO '레이븐2' 29일 오후 8시 출시 확정,사진=<b>넷마블</b> 제공 <b>넷마블</b>은 블록버스터 신작 MMORPG '...,https://www.techm.kr/news/articleView.html?idx...,https://www.techm.kr/news/articleView.html?idx...,2024-05-22 11:44:00,넷마블
1997,1998,[#게임스테이션] '레이븐2' 29일 오후 8시 출시·'퍼스트 디센던트' 소개영상...,<b>넷마블</b>이 '레이븐2'를 오는 29일 오후 8시로 출시하고 넥슨게임즈가 ...,https://www.newsworks.co.kr/news/articleView.h...,https://www.newsworks.co.kr/news/articleView.h...,2024-05-22 11:40:00,넷마블
1998,1999,"<b>넷마블</b>문화재단, '게임아카데미 9기' 정규과정 발대식 진행",<b>넷마블</b>문화재단(이사장 방준혁)은 지난 21일 <b>넷마블</b> 사옥에...,http://www.nextdaily.co.kr/news/articleView.ht...,http://www.nextdaily.co.kr/news/articleView.ht...,2024-05-22 11:36:00,넷마블


In [26]:
# SQL 쿼리 작성 (예: 'SELECT * FROM your_table')
query = 'SELECT * FROM stock_prices'

# SQL 쿼리를 실행하고 결과를 DataFrame으로 변환
df = pd.read_sql_query(query, conn)